### Import Libraries and Data

In [16]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os

### LOAD DATA
umsatz_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/umsatzdaten_gekuerzt.csv"
wetter_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/wetter.csv"
kiwo_url = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/kiwo.csv"

# Load external data
umsatz_df = pd.read_csv(umsatz_url)
wetter_df = pd.read_csv(wetter_url)
kiwo_df   = pd.read_csv(kiwo_url)
test_df   = pd.read_csv("test.csv",  parse_dates=["Datum"])

#print headers of the dataframes
print("Umsatz DataFrame:")
display(umsatz_df.head())
print("\nWetter DataFrame:")
display(wetter_df.head())
print("\nKiwo DataFrame:")
display(kiwo_df.head())
print("\nTest DataFrame:")
display(test_df.head())

Umsatz DataFrame:


,id,Datum,Warengruppe,Umsatz
0,1307011,2013-07-01,1,148.828353
1,1307021,2013-07-02,1,159.793757
2,1307031,2013-07-03,1,111.885594
3,1307041,2013-07-04,1,168.864941
4,1307051,2013-07-05,1,171.280754



Wetter DataFrame:


,Datum,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode
0,2012-01-01,8.0,9.8250,14,58.0
1,2012-01-02,7.0,7.4375,12,NaN
2,2012-01-03,8.0,5.5375,18,63.0
3,2012-01-04,4.0,5.6875,19,80.0
4,2012-01-05,6.0,5.3000,23,80.0



Kiwo DataFrame:


,Datum,KielerWoche
0,2012-06-16,1
1,2012-06-17,1
2,2012-06-18,1
3,2012-06-19,1
4,2012-06-20,1



Test DataFrame:


,id,Datum,Warengruppe
0,1808011,2018-08-01,1
1,1808021,2018-08-02,1
2,1808031,2018-08-03,1
3,1808041,2018-08-04,1
4,1808051,2018-08-05,1


### Merging Data

In [17]:
### MERGE DATA
merged_df = pd.concat([umsatz_df, test_df], axis=0, ignore_index=True)
merged_df = pd.merge(merged_df, wetter_df, on="Datum", how="left")
merged_df = pd.merge(merged_df, kiwo_df, on="Datum", how="left")
merged_df["Datum"] = pd.to_datetime(merged_df["Datum"])
merged_df = merged_df.sort_values('Datum')

# print headers of the dataframes
print("Merged DataFrame:")
display(merged_df.head())

Merged DataFrame:


,id,Datum,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche
0,1307011,2013-07-01,1,148.828353,6.0,17.8375,15.0,20.0,NaN
3638,1307013,2013-07-01,3,201.198426,6.0,17.8375,15.0,20.0,NaN
5457,1307014,2013-07-01,4,65.890169,6.0,17.8375,15.0,20.0,NaN
7223,1307015,2013-07-01,5,317.475875,6.0,17.8375,15.0,20.0,NaN
1819,1307012,2013-07-01,2,535.856285,6.0,17.8375,15.0,20.0,NaN


### Data Cleaning + Imputing

In [18]:
### MISSING VALUE HANDLING
merged_df = merged_df.set_index("Datum")
merged_df["Temperatur"] = merged_df["Temperatur"].interpolate(method="time")
merged_df["Windgeschwindigkeit"] = merged_df["Windgeschwindigkeit"].interpolate(method="time")
merged_df["Bewoelkung"] = merged_df["Bewoelkung"].interpolate(method="time")
merged_df["KielerWoche"] = merged_df["KielerWoche"].fillna(0)
merged_df["Wettercode"] = merged_df["Wettercode"].fillna(method="ffill").fillna(method="bfill")
merged_df = merged_df.reset_index()

# print headers of the dataframes
print("Merged DataFrame:")
display(merged_df.head())

Merged DataFrame:


/tmp/ipykernel_80803/284560231.py:7: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df["Wettercode"] = merged_df["Wettercode"].fillna(method="ffill").fillna(method="bfill")


,Datum,id,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche
0,2013-07-01,1307011,1,148.828353,6.0,17.8375,15.0,20.0,0.0
1,2013-07-01,1307013,3,201.198426,6.0,17.8375,15.0,20.0,0.0
2,2013-07-01,1307014,4,65.890169,6.0,17.8375,15.0,20.0,0.0
3,2013-07-01,1307015,5,317.475875,6.0,17.8375,15.0,20.0,0.0
4,2013-07-01,1307012,2,535.856285,6.0,17.8375,15.0,20.0,0.0


### Data Preparation

In [19]:
### DEFINE CATEGORICAL VARIABLES
wetter_dummies = pd.get_dummies(merged_df["Wettercode"].astype(int), prefix="WetterCode").astype(int)
merged_df = pd.concat([merged_df, wetter_dummies], axis=1)

warengruppe_dummies = pd.get_dummies(merged_df["Warengruppe"], prefix="Warengruppe").astype(int)
merged_df = pd.concat([merged_df, warengruppe_dummies], axis=1)

dummy_cols = list(wetter_dummies.columns) + list(warengruppe_dummies.columns)
merged_df[dummy_cols] = merged_df[dummy_cols].astype(int)

print("Merged DataFrame:")
display(merged_df.head())

Merged DataFrame:


,Datum,id,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,WetterCode_0,...,WetterCode_75,WetterCode_77,WetterCode_79,WetterCode_95,Warengruppe_1,Warengruppe_2,Warengruppe_3,Warengruppe_4,Warengruppe_5,Warengruppe_6
0,2013-07-01,1307011,1,148.828353,6.0,17.8375,15.0,20.0,0.0,0,...,0,0,0,0,1,0,0,0,0,0
1,2013-07-01,1307013,3,201.198426,6.0,17.8375,15.0,20.0,0.0,0,...,0,0,0,0,0,0,1,0,0,0
2,2013-07-01,1307014,4,65.890169,6.0,17.8375,15.0,20.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
3,2013-07-01,1307015,5,317.475875,6.0,17.8375,15.0,20.0,0.0,0,...,0,0,0,0,0,0,0,0,1,0
4,2013-07-01,1307012,2,535.856285,6.0,17.8375,15.0,20.0,0.0,0,...,0,0,0,0,0,1,0,0,0,0


### Define additional Variables

In [20]:
# ### take sales from last week
# merged_df["sales_last_week"] = merged_df["Umsatz"].shift(7)
# merged_df["sales_last_week"] = merged_df["sales_last_week"].fillna(0)
# merged_df["sales_last_week"] = merged_df["sales_last_week"].astype(float)

# ### TAKE SALES FROM LAST YEAR
# merged_df["sales_last_year"] = merged_df["Umsatz"].shift(365)
# merged_df["sales_last_year"] = merged_df["sales_last_year"].fillna(0)
# merged_df["sales_last_year"] = merged_df["sales_last_year"].astype(float)
# merged_df = merged_df.dropna(subset=["sales_last_week", "sales_last_year"])

### COMBINE WARENGRUPPEN SALES
sales_daily_df = merged_df.groupby("Datum")["Umsatz"].sum().reset_index()
sales_daily_df = sales_daily_df.rename(columns={"Umsatz": "sales_daily"})
merged_df = pd.merge(merged_df, sales_daily_df, on="Datum", how="left")

 
### FEIERTAGE
import holidays
de_holidays = holidays.Germany(years=range(2012, 2021), subdiv='SH')
hol_df = pd.DataFrame(list(de_holidays.items()), columns=["Datum", "Feiertag"])
hol_df["Datum"] = pd.to_datetime(hol_df["Datum"]).dt.normalize()
hol_df["is_holiday"] = 1

relevante_feiertage = ["Erster Weihnachtstag", "Zweiter Weihnachtstag", "Karfreitag", "Ostersonntag", "Ostermontag", "Neujahr"]
vortage = hol_df[hol_df["Feiertag"].isin(relevante_feiertage)].copy()
vortage["Datum"] = vortage["Datum"] - pd.Timedelta(days=1)
vortage["Feiertag"] = vortage["Feiertag"] + " (Vortag)"

hol_df_erweitert = pd.concat([hol_df, vortage], ignore_index=True)
hol_df_erweitert = hol_df_erweitert.drop_duplicates(subset="Datum")

merged_df["Datum"] = pd.to_datetime(merged_df["Datum"]).dt.normalize()
merged_df = pd.merge(merged_df, hol_df_erweitert[["Datum", "Feiertag"]], on="Datum", how="left")
merged_df["Feiertag"] = merged_df["Feiertag"].fillna("Kein Feiertag")
merged_df["is_holiday"] = (merged_df["Feiertag"] != "Kein Feiertag").astype(int)

hol_onehot = hol_df.assign(Wert=1).pivot_table(index="Datum", columns="Feiertag", values="Wert", fill_value=0).reset_index()
merged_df["Datum"] = pd.to_datetime(merged_df["Datum"]).dt.normalize()
merged_df = merged_df.merge(hol_onehot, on="Datum", how="left")
feiertagsspalten = hol_onehot.columns.drop("Datum")
merged_df[feiertagsspalten] = merged_df[feiertagsspalten].fillna(0).astype(int)


### DAX
import yfinance as yf1
dax_df = yf1.download("^GDAXI", start="2012-01-01", end="2020-12-31")["Close"]
dax_df = (dax_df.reset_index().rename(columns={'Date': 'Datum','Close': 'DAX_Close'}))
dax_df = dax_df.rename(columns={"^GDAXI": "DAX"})
merged_df = merged_df.merge(dax_df, on="Datum", how="left")
merged_df["DAX"] = (merged_df["DAX"].fillna(method="ffill").fillna(method="bfill"))


### WOCHENTAG
merged_df["Wochentag"] = merged_df["Datum"].dt.weekday
wochentag_dummies = pd.get_dummies(merged_df["Wochentag"], prefix="weekday")
merged_df = pd.concat([merged_df, wochentag_dummies], axis=1)
dummy_cols = list(wochentag_dummies.columns) 
merged_df[dummy_cols] = merged_df[dummy_cols].astype(int)


### SUNHOURS
from astral import sun
from astral import Observer
obs = Observer(latitude=54.3233, longitude=10.1228)
merged_df['day_length_h'] = merged_df['Datum'].dt.date.apply(lambda d: (sun.sun(observer=obs, date=d, tzinfo=None)['sunset'] - sun.sun(observer=obs, date=d, tzinfo=None)['sunrise']).total_seconds() / 3600)


### FUSSBALL!!!
def fussball_flag(datum):
    em_2016 = pd.Timestamp('2016-06-10') <= datum <= pd.Timestamp('2016-07-10')
    em_2021 = pd.Timestamp('2021-06-11') <= datum <= pd.Timestamp('2021-07-11')
    wm_2014 = pd.Timestamp('2014-06-12') <= datum <= pd.Timestamp('2014-07-13')
    wm_2018 = pd.Timestamp('2018-06-14') <= datum <= pd.Timestamp('2018-07-15')
    return int(em_2016 or em_2021 or wm_2014 or wm_2018)
merged_df["fussball_event"] = merged_df["Datum"].apply(fussball_flag)


### SCHULFERIEN
ferien_liste = [
    # 2013
    {"name": "Winterferien", "start": "2013-01-31", "end": "2013-02-01"},
    {"name": "Osterferien", "start": "2013-03-25", "end": "2013-04-09"},
    {"name": "Sommerferien", "start": "2013-06-24", "end": "2013-08-03"},
    {"name": "Herbstferien", "start": "2013-10-04", "end": "2013-10-18"},
    {"name": "Weihnachtsferien", "start": "2013-12-23", "end": "2014-01-06"},

    # 2014
    {"name": "Winterferien", "start": "2014-01-31", "end": "2014-01-31"},
    {"name": "Osterferien", "start": "2014-04-16", "end": "2014-05-02"},
    {"name": "Sommerferien", "start": "2014-07-14", "end": "2014-08-23"},
    {"name": "Herbstferien", "start": "2014-10-13", "end": "2014-10-25"},
    {"name": "Weihnachtsferien", "start": "2014-12-22", "end": "2015-01-06"},

    # 2015
    {"name": "Winterferien", "start": "2015-01-30", "end": "2015-01-30"},
    {"name": "Osterferien", "start": "2015-03-30", "end": "2015-04-17"},
    {"name": "Sommerferien", "start": "2015-07-20", "end": "2015-08-29"},
    {"name": "Herbstferien", "start": "2015-10-19", "end": "2015-10-31"},
    {"name": "Weihnachtsferien", "start": "2015-12-21", "end": "2016-01-06"},

    # 2016
    {"name": "Winterferien", "start": "2016-02-01", "end": "2016-02-02"},
    {"name": "Osterferien", "start": "2016-03-21", "end": "2016-04-02"},
    {"name": "Sommerferien", "start": "2016-07-25", "end": "2016-09-03"},
    {"name": "Herbstferien", "start": "2016-10-17", "end": "2016-10-29"},
    {"name": "Weihnachtsferien", "start": "2016-12-23", "end": "2017-01-06"},

    # 2017
    {"name": "Winterferien", "start": "2017-01-30", "end": "2017-01-31"},
    {"name": "Osterferien", "start": "2017-04-03", "end": "2017-04-18"},
    {"name": "Sommerferien", "start": "2017-07-24", "end": "2017-09-02"},
    {"name": "Herbstferien", "start": "2017-10-16", "end": "2017-10-27"},
    {"name": "Weihnachtsferien", "start": "2017-12-21", "end": "2018-01-06"},

    # 2018
    {"name": "Winterferien", "start": "2018-02-01", "end": "2018-02-02"},
    {"name": "Osterferien", "start": "2018-03-29", "end": "2018-04-21"},
    {"name": "Sommerferien", "start": "2018-07-09", "end": "2018-08-18"},
    {"name": "Herbstferien", "start": "2018-10-01", "end": "2018-10-19"},
    {"name": "Weihnachtsferien", "start": "2018-12-21", "end": "2019-01-06"},

    # 2019
    {"name": "Winterferien", "start": "2019-01-31", "end": "2019-02-01"},
    {"name": "Osterferien", "start": "2019-04-04", "end": "2019-04-18"},
    {"name": "Sommerferien", "start": "2019-06-28", "end": "2019-08-07"},
    {"name": "Herbstferien", "start": "2019-10-04", "end": "2019-10-18"},
    {"name": "Weihnachtsferien", "start": "2019-12-23", "end": "2020-01-06"},

    # 2020
    {"name": "Winterferien", "start": "2020-01-31", "end": "2020-01-31"},
    {"name": "Osterferien", "start": "2020-04-06", "end": "2020-04-18"},
    {"name": "Sommerferien", "start": "2020-06-29", "end": "2020-08-08"},
    {"name": "Herbstferien", "start": "2020-10-05", "end": "2020-10-17"},
    {"name": "Weihnachtsferien", "start": "2020-12-21", "end": "2021-01-06"},

    # 2021
    {"name": "Winterferien", "start": "2021-02-01", "end": "2021-02-02"},
    {"name": "Osterferien", "start": "2021-04-01", "end": "2021-04-16"},
    {"name": "Sommerferien", "start": "2021-06-21", "end": "2021-07-31"},
    {"name": "Herbstferien", "start": "2021-10-04", "end": "2021-10-16"},
    {"name": "Weihnachtsferien", "start": "2021-12-23", "end": "2022-01-07"},
]

# Ferien-Flag initialisieren
merged_df["schulferien"] = 0
merged_df["ferientyp"] = "Keine Ferien"

df_ferien = pd.DataFrame(ferien_liste)
df_ferien["start"] = pd.to_datetime(df_ferien["start"])
df_ferien["end"] = pd.to_datetime(df_ferien["end"])

for _, row in df_ferien.iterrows():
    maske = (merged_df["Datum"] >= row["start"]) & (merged_df["Datum"] <= row["end"])
    merged_df.loc[maske, "schulferien"] = 1
    merged_df.loc[maske, "ferientyp"] = row["name"]

ferien_dummies = pd.get_dummies(merged_df["ferientyp"], prefix="Ferien")
merged_df = pd.concat([merged_df, ferien_dummies], axis=1)
dummy_cols = list(ferien_dummies.columns)
merged_df[dummy_cols] = merged_df[dummy_cols].astype(int)

#
merged_df["Monat"] = merged_df["Datum"].dt.month
merged_df["Wochentag"] = merged_df["Datum"].dt.weekday

# Durchschnittlicher Umsatz pro Warengruppe, Monat, Wochentag berechnen
avg_sales = (merged_df.groupby(["Warengruppe", "Monat", "Wochentag"])["Umsatz"].mean().reset_index().rename(columns={"Umsatz": "avg_sales_per_group_month_weekday"}))




# Merge zurück ins merged_df (auf die passenden Zeilen)
merged_df = pd.merge(merged_df, avg_sales, on=["Warengruppe", "Monat", "Wochentag"], how="left")



print("Merged DataFrame:")
display(merged_df.head())


/tmp/ipykernel_80803/1209176653.py:47: FutureWarning: YF.download() has changed argument auto_adjust default to True
  dax_df = yf1.download("^GDAXI", start="2012-01-01", end="2020-12-31")["Close"]
[*********************100%***********************]  1 of 1 completed
/tmp/ipykernel_80803/1209176653.py:51: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df["DAX"] = (merged_df["DAX"].fillna(method="ffill").fillna(method="bfill"))


Merged DataFrame:


,Datum,id,Warengruppe,Umsatz,Bewoelkung,Temperatur,Windgeschwindigkeit,Wettercode,KielerWoche,WetterCode_0,...,schulferien,ferientyp,Ferien_Herbstferien,Ferien_Keine Ferien,Ferien_Osterferien,Ferien_Sommerferien,Ferien_Weihnachtsferien,Ferien_Winterferien,Monat,avg_sales_per_group_month_weekday
0,2013-07-01,1307011,1,148.828353,6.0,17.8375,15.0,20.0,0.0,0,...,1,Sommerferien,0,0,0,1,0,0,7,156.363405
1,2013-07-01,1307013,3,201.198426,6.0,17.8375,15.0,20.0,0.0,0,...,1,Sommerferien,0,0,0,1,0,0,7,239.528780
2,2013-07-01,1307014,4,65.890169,6.0,17.8375,15.0,20.0,0.0,0,...,1,Sommerferien,0,0,0,1,0,0,7,81.701945
3,2013-07-01,1307015,5,317.475875,6.0,17.8375,15.0,20.0,0.0,0,...,1,Sommerferien,0,0,0,1,0,0,7,297.246705
4,2013-07-01,1307012,2,535.856285,6.0,17.8375,15.0,20.0,0.0,0,...,1,Sommerferien,0,0,0,1,0,0,7,532.285325


### Splitting Data

In [21]:
import pandas as pd
from datetime import datetime

# Convert Datum column to datetime if not already done
merged_df['Datum'] = pd.to_datetime(merged_df['Datum'])

# Define date ranges for splitting
train_start = datetime(2013, 7, 1)
train_end = datetime(2017, 7, 31)
val_start = datetime(2017, 8, 1)
val_end = datetime(2018, 7, 31)
test_start = datetime(2018, 8, 1)
test_end = datetime(2019, 7, 31)

# Split the dataset based on date ranges
train_set = merged_df[(merged_df['Datum'] >= train_start) & 
                      (merged_df['Datum'] <= train_end)].copy()

validation_set = merged_df[(merged_df['Datum'] >= val_start) & 
                          (merged_df['Datum'] <= val_end)].copy()

test_set = merged_df[(merged_df['Datum'] >= test_start) & 
                     (merged_df['Datum'] <= test_end)].copy()

### Store Data

In [22]:
### STORE DATA
merged_df.to_csv("../0_DataPreparation/merged_output.csv", index=False)

train_set.to_csv("../0_DataPreparation/train_set.csv", index=False, mode='w')
validation_set.to_csv("../0_DataPreparation/validation_set.csv", index=False, mode='w')
test_set.to_csv("../0_DataPreparation/test_set.csv", index=False, mode='w')